---

# Periodic systems

Exercises for periodic systems, like an accelerator ring (exercises 18-27).

---

In [1]:
# Import custom tracking functions and useful libraries
from tracking_library import *

# Set up plotting
%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

### Exercise 18
Try to find an initial beam matrix sigma0 by **try and error** that reproduces itself at the end of the beam line (5 FODO cells). You will quickly realise that this is extremely difficult if not impossible. 

**Hint 1**: (optional) you might want to use an **interactive plot** - see in the previous exercises how to make one

**Hint 2**: try, but **don't waist to much time with try and error** approaches (that's not your job as accelerator physicist!), and **move on** to the next exercise...

#### SOLUTION

In [2]:
# code here your solution...





### Exercise 19

Try to simplify the problem of previous exercise. **Recall** [Wolfgan's lecture](https://indico.cern.ch/event/1117526/contributions/4978192/):

<p align="center">
<img src=./_img_exercises/slide_fodo_periodic.png width="60%">
</p>

In a FODO cell the $\beta$ function reaches a maximum (or a minimum) in the middle of the quadrupoles, i.e. the $\alpha$ function is zero.
If one models the FODO cell starting from the middle of a quadrupole (instead of the middle of a drift), then at least one parameter is fixed.

Even better: one can use the equation form [Wolfgan's lecture](https://indico.cern.ch/event/1117526/contributions/4978192/) to find the theoretical solution.

**Hint**: A half-quadrupole has double the focal length than a full quadrupole.

#### SOLUTION

In [3]:
# code here your solution...







## General solution

**Recall** [Wolfgan's lecture](https://indico.cern.ch/event/1117526/contributions/4978192/). In a periodic beam line, the transfer matrix assumes the form:

\begin{equation}
M = 
    \left[
    \begin{array}{cc}
    \cos(\mu) + \alpha_0 \sin(\mu)  & \beta_0 \sin(\mu)\\
    - \gamma_0 \sin(\mu) & \cos(\mu) - \alpha_0 \sin(\mu)
    \end{array}
    \right]
\end{equation}

where $\alpha_0$, $\beta_0$, $\gamma_0$ are the initial (and final) Twiss values and $\mu$ is the phase advance (or tune, if we talk about a whole ring) of the associated beamline.
From this expression, one can extract the periodic solution of any beamline, assuming it exists!

Note: Since the determinant of $M$ must be one, then one can verify that the trace of the matrix R must be $|\mathrm{Tr}(M)| \le 2$ to ensure the existence of such a solution.

This can be implemented in an useful function:

In [4]:
def twiss(beamline):
    '''
    Computes and returns the closed solution (if it exist!) Tune (Q) and Twiss parameters 
    beta, alpha, gamma of the given beamline.
    '''

    # first, compute the equivalent "One-Turn-Map", and extract its matrix:
    OTM = getEquivalentElement(beamline)
    R = OTM[0]['matrix']
    
    # check that this matrix is stable:
    if np.abs(0.5*(R[0,0]+R[1,1])) > 1:
        raise ValueError('This beamline is not stable!')
    
    # all relevant Twiss parameters can be extrcted from the matrix:
    mu = np.arccos(0.5*(R[0,0]+R[1,1]))
    if (R[0,1]<0): 
        mu = 2*np.pi-mu
    Q = mu/(2*np.pi)
    beta = R[0,1]/np.sin(mu)
    alpha = (0.5*(R[0,0]-R[1,1]))/np.sin(mu)
    gamma = (1+alpha**2)/beta
    
    return Q, beta, alpha, gamma

### Exercise 20

Let's go back to our initial FODO cell, starting from the middle of a drift.
Find the range of focal lengths $f$ for which the FODO cells permit stable oscillations based on the newly introduced condition (i.e. $|Tr(R)| \le 2$)

#### SOLUTION


In [5]:
# code here your solution...






### Exercise 21

Consider our usual FODO cell starting from the middle of a drift, and which allows for having a periodic solution (i.e. $f > L_{FODO}/4$)
Compute the periodic $\sigma_0$ matrix using the Twiss parameters obtain for the newly defined `twiss` function (assuming $\epsilon=1$).

Transport the periodic $\Sigma$ matrix along the FODO and convince yourself that the $\Sigma_s$ matrix at the end of the FODO is indeed equal to the one at the start, $\Sigma_0$.

#### SOLUTION

In [6]:
# code here your solution...





### Exercise 22
Write down the numerical values of initial beam matrix $\Sigma_0$, then build a beam line made of $15$ consecutive cells by changing the definition of the lattice and then, using $\Sigma_0$ with the noted-down numbers, prepare a plot of the beam sizes along the $15$ cells. 
Is this also periodic?

#### SOLUTION

In [7]:
# code here your solution...





### Exercise 23

Consider our usual FODO cell.
Consider a single particle with some non-zero initial $x_0$ and/or $x_0'$, and plot the $x$, $x'$ coordinates at the end of each FODO cell for a beamline made of 100 cells. 

**Hint:** You should use both `getEquivalentElement` to obtain the single FODO cell element, and `transportParticles` function to track the particle over several of such a cell.

**Hint:** use `plt.scatter(output['x'], output['px'])` to plot all phase-space coordinates.

#### SOLUTIONS
For plotting the trace space evolution of a particle we need to observe it turn-after-turn. The natural way to do that is to compress the beam line (in general composed by more than one element) in a single transformation: the one-turn-matrix, which can be computed using the `getEquivalentElement` function previously introduced.

In [8]:
# code here your solution...





### Exercise 24 (optional)
In the definition of FODO of the previous exercise, reverse the polarity of both quadrupoles and prepare a phase-space plot. How does it differ from the one in Exercise 18?

#### SOLUTION

In [9]:
# code here your solution...






---

## Ellipses ans Single Particle Emittance

It turns out that our particle turn-after-turn stays on an ellipses: 

\begin{equation}
\epsilon = \gamma x^2 + 2 \alpha x x' + \beta x'\,^2
\end{equation}

where $(\alpha, \beta, \gamma)$ are the periodic Twiss parameter! and $\epsilon$, in this case, is the **single particle emittance**.

Without entering in the implementation details, let's define two useful functions that can give us the single particle emittance (`particle_emittance(x, xp, beta, alpha)`) or to give us the coordinate in phase space for an ellipse with the given Twiss and emittance (`ellipse_points(emittance, beta, alpha, n_points = 100)`)

In [10]:
def particle_emittance(x, xp, beta, alpha):
    '''Returns the single particle emittance for a given 
       x, xp particle coordinates and assumed beta and alpha parameters
    '''
    # compute gamma
    gamma = (alpha**2 + 1)/beta
    # compute and return the associated single particle emittance
    epsilon = gamma*x**2 + 2*alpha*x*xp + beta*xp**2
    return epsilon

def ellipse_points(emittance, beta, alpha, n_points = 100):
    ''' Returns the x,x' coordinates of an ellipse in phase space for 
        the given Twiss parameters (beta, gamma, emittance)
    '''
    # generate a uniform sampling of phases:
    thetas = np.linspace(0, 2*np.pi, n_points)
    # generate coordinates
    x  = np.sqrt(emittance*beta)*np.cos(thetas)
    xp = -np.sqrt(emittance/beta)*(alpha*np.cos(thetas)-np.sin(thetas))
    # return them in our usual form
    return np.array([x, xp])


### Exercise 25

Consider the usual FODO cell, repeated 100 times.
Plot the trajectory, in phase space, of a particle with x=1 mm and xp=0 cell after cell.

Using the functions you have now available, compute the expected periodic solution of our cell, and use this information to:

- compute the single particle emittance
- produce the coordinates in phase space of an ellipse with such an emittance

Plot both the tracked particle coordinates and the ellipse in phase space: what do you observe?

**Optional:** would you be able to predict the exact tracked particle coordinates only from the single cell?

**Hint:** for this latter question, one needs to look carefully at the functions implementation details...


In [11]:
# code here your solution...





### Exercise 26
Plot the position of the particle vs the number of turns. 

1. What do you observe?
2. Can you infer the tune? 
3. How many turns (or FODO cells) are needed for the particle to complete one full oscillation?
4. What changes if you start from a different initial coordinate?

#### SOLUTION

In [12]:
# code here your solution...





### Exercise 27

Consider a single FODO cell of total length $L_{FODO}=2$ m. 

1. Can you find $f$ such that the FODO cell phase advance is $\mu = \pi/2$, i.e. 90 deg phase-advance?
2. Repeat this cell 100 times, and plot the trajectory, in phase space, of a particle with x=1 mm and xp=0 cell after cell: what do you observe?
3. (Optional) Show that the similar results are obtained irrespectively if you start your FODO from the middle of a quadrupole or of a drift.

**Hint:** Look back at [Wolfgan's lecture](https://indico.cern.ch/event/1117526/contributions/4978192/):

<p align="center">
<img src=./_img_exercises/slide_fodo_phase.png width="60%">
</p>

and recall that $\mathrm{Tr}(M) = 2 \cos(\mu)$. 
With some math, one can prove that:

\begin{equation}
\sin(\mu/2) = \frac{L_{FODO}}{4f}
\end{equation}



#### SOLUTION

In [13]:
# code here your solution...




